In [1]:
import os
import pandas as pd
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import gc
import numpy as np
from keras import backend as K
from sklearn.metrics import fbeta_score
from keras.layers import Conv2D, Dense, LSTM, Flatten, MaxPooling2D, BatchNormalization, Dropout
from keras.optimizers import Adam, RMSprop, SGD
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from sklearn.metrics import fbeta_score
from tqdm import tqdm
from sklearn.utils import shuffle
import cv2
from PIL import Image
from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Sequential 
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from keras.optimizers import Adam, SGD
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, History

In [2]:
#reading the labels
train_label = pd.read_csv('../input/planets-dataset/planet/planet/train_classes.csv')
sam_sub = pd.read_csv('../input/planets-dataset/planet/planet/sample_submission.csv')
train_label.head()

,image_name,tags
0,train_0,haze primary
1,train_1,agriculture clear primary water
2,train_2,clear primary
3,train_3,clear primary
4,train_4,agriculture clear habitation primary road


In [3]:
#onehot=OneHotEncoder()
encoder = LabelEncoder()

In [4]:
#encoder.fit(train_label['tags'])
label_maps = pd.DataFrame()
label_maps['tags'] = ['agriculture', 'artisinal_mine', 'bare_ground','blooming','blow_down','clear','cloudy','conventional_mine','cultivation','habitation','haze', 'partly_cloudy','primary','road','selective_logging','slash_burn','water']
label_maps['map'] = encoder.fit_transform(label_maps['tags'])
label_maps

,tags,map
0,agriculture,0
1,artisinal_mine,1
2,bare_ground,2
3,blooming,3
4,blow_down,4
5,clear,5
6,cloudy,6
7,conventional_mine,7
8,cultivation,8
9,habitation,9


In [5]:
#defining a dict of encoded labels
label_map = {'agriculture': 0,
 'artisinal_mine': 1,
 'bare_ground': 2,
 'blooming': 3,
 'blow_down': 4,
 'clear': 5,
 'cloudy': 6,
 'conventional_mine': 7,
 'cultivation': 8,
 'habitation': 9,
 'haze': 10,
 'partly_cloudy': 11,
 'primary': 12,
 'road': 13,
 'selective_logging': 14,
 'slash_burn': 15,
 'water': 16}


In [6]:
#loading the traing_images
X = []
Y = []
train_label = shuffle(train_label,random_state=0)

In [7]:
for image_name, tags in tqdm(train_label.values, miniters=400):
    arr = cv2.imread('../input/planets-dataset/planet/planet/train-jpg/{}.jpg'.format(image_name), cv2.IMREAD_UNCHANGED)
    targets = np.zeros(17)
    for t in tags.split(' '):
      targets[label_map[t]] = 1 
    arr = cv2.resize(arr, (64, 64))
    X.append(arr)
    Y.append(targets)

100%|██████████| 40479/40479 [04:35<00:00, 146.87it/s]


In [8]:
X = np.array(X, np.float16)/255.0

In [9]:
#splitting
X = np.array(X)
Y = np.array(Y)
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size = 0.2, shuffle = True, random_state = 1)

print(x_train.shape, y_train.shape, x_val.shape, y_val.shape)

(32383, 64, 64, 3) (32383, 17) (8096, 64, 64, 3) (8096, 17)


In [10]:
#deleting uneccessary datas
del X, Y
import gc
gc.collect()

132

In [11]:
import gc
gc.collect()

22

In [12]:
def fbeta(y_true, y_pred, threshold_shift=0):
    beta = 2

    # just in case of hipster activation at the final layer
    y_pred = K.clip(y_pred, 0, 1)

    # shifting the prediction threshold from .5 if needed
    y_pred_bin = K.round(y_pred + threshold_shift)

    tp = K.sum(K.round(y_true * y_pred_bin)) + K.epsilon()
    fp = K.sum(K.round(K.clip(y_pred_bin - y_true, 0, 1)))
    fn = K.sum(K.round(K.clip(y_true - y_pred, 0, 1)))

    precision = tp / (tp + fp)
    recall = tp / (tp + fn)

    beta_squared = beta ** 2
    return (beta_squared + 1) * (precision * recall) / (beta_squared * precision + recall + K.epsilon())

In [13]:
#with tpu_strategy.scope():
model = keras.Sequential()
model.add(Conv2D(64, 5, 2, activation = "relu", input_shape = (64, 64, 3)))
model.add(MaxPooling2D())
model.add(Conv2D(128, 5, 2, activation = "relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(512, activation = "relu"))
model.add(Dense(17, activation = "sigmoid"))
model.compile(loss = "binary_crossentropy", optimizer = Adam(), metrics = [fbeta])
model.fit(x_train, y_train, validation_data = (x_val, y_val), epochs = 45, batch_size = 64)

Epoch 1/45
506/506 [==============================] - 4s 7ms/step - loss: 0.2254 - fbeta: 0.6387 - val_loss: 0.2024 - val_fbeta: 0.7036
Epoch 2/45
506/506 [==============================] - 3s 5ms/step - loss: 0.1965 - fbeta: 0.6849 - val_loss: 0.1855 - val_fbeta: 0.7130
Epoch 3/45
506/506 [==============================] - 3s 6ms/step - loss: 0.1784 - fbeta: 0.7200 - val_loss: 0.1728 - val_fbeta: 0.7045
Epoch 4/45
506/506 [==============================] - 3s 5ms/step - loss: 0.1639 - fbeta: 0.7459 - val_loss: 0.1635 - val_fbeta: 0.7712
Epoch 5/45
506/506 [==============================] - 3s 6ms/step - loss: 0.1550 - fbeta: 0.7621 - val_loss: 0.1508 - val_fbeta: 0.7839
Epoch 6/45
506/506 [==============================] - 3s 6ms/step - loss: 0.1500 - fbeta: 0.7705 - val_loss: 0.1467 - val_fbeta: 0.7747
Epoch 7/45
506/506 [==============================] - 3s 6ms/step - loss: 0.1450 - fbeta: 0.7782 - val_loss: 0.1445 - val_fbeta: 0.7766
Epoch 8/45
506/506 [============================

In [14]:
import gc
gc.collect()

1555

In [15]:
#with tpu_strategy.scope():
test_loss, test_accuracy = model.evaluate(x_val, y_val)
print('Test loss: {}'.format(test_loss))
print('Test accuracy: {}'.format(test_accuracy))

253/253 [==============================] - 1s 2ms/step - loss: 0.2290 - fbeta: 0.8059
Test loss: 0.2289707213640213
Test accuracy: 0.8059197068214417


In [16]:
import gc
gc.collect()

743

In [17]:
model.evaluate(x_val, y_val)
print("train fscore: ", fbeta_score(y_train, np.round(model.predict(x_train)), 2,average = 'weighted'))
print("val fscore: ", fbeta_score(y_val, np.round(model.predict(x_val)), 2, average = 'weighted'))

253/253 [==============================] - 1s 2ms/step - loss: 0.2290 - fbeta: 0.8059


/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:70: FutureWarning: Pass beta=2 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


train fscore:  0.9321623395315798
val fscore:  0.800504961104014


In [18]:
import gc
del x_train, y_train, x_val, y_val
gc.collect()

1413

In [19]:
#dividing my test_labels into two part for test-jpg and test-jpg-additional
test = sam_sub[0 : 40669]
files = sam_sub[40669 : ]

In [20]:
#with tpu_strategy.scope():   
test_img = []

for image_name, tags in tqdm(test.values, miniters=1000):
    arr = cv2.imread('../input/planets-dataset/planet/planet/test-jpg/{}.jpg'.format(image_name))
    test_img.append(cv2.resize(arr, (64, 64)))

for image_name, tags in tqdm(files.values, miniters=1000):
    arr = cv2.imread('../input/planets-dataset/test-jpg-additional/test-jpg-additional/{}.jpg'.format(image_name))
    test_img.append(cv2.resize(arr, (64, 64)))

test_img = np.array(test_img, np.float16)/255.0

100%|██████████| 20522/20522 [02:14<00:00, 152.83it/s]


In [21]:
import gc
gc.collect()

22

In [22]:
#with tpu_strategy.scope():
yres = []
predictions = model.predict(test_img, batch_size = 64, verbose = 2)
yres.append(predictions)

957/957 - 3s


In [23]:
import gc
gc.collect()

743

In [24]:
#converting my encoded labels back to it original form
sub = np.array(yres[0])
for i in range (1, len(yres)):
    sub += np.array(yres[i])
sub = pd.DataFrame(sub, columns = label_map)


In [25]:
sub

,agriculture,artisinal_mine,bare_ground,blooming,blow_down,clear,cloudy,conventional_mine,cultivation,habitation,haze,partly_cloudy,primary,road,selective_logging,slash_burn,water
0,0.026761,1.804929e-11,2.975968e-04,4.393901e-02,1.698533e-03,9.999774e-01,4.124584e-07,1.146589e-11,1.617980e-03,3.193776e-03,1.743644e-04,1.214020e-06,0.999999,0.006030,3.713174e-03,4.863562e-09,0.001178
1,0.139083,4.994773e-09,8.398611e-04,4.946046e-04,1.321725e-03,9.913284e-01,1.918780e-07,8.904638e-09,2.978957e-02,1.496234e-01,9.853930e-05,1.438168e-02,0.999998,0.004471,1.326626e-02,6.467887e-05,0.296430
2,0.099209,4.959773e-12,4.550067e-12,3.602277e-23,1.277810e-15,1.431465e-14,2.347315e-10,2.375387e-14,8.016458e-04,4.641850e-10,1.594450e-17,1.000000e+00,1.000000,0.000226,5.511212e-20,3.914499e-18,0.077023
3,0.256978,3.399984e-10,1.113010e-04,1.136076e-02,3.120139e-03,6.294459e-01,1.325748e-08,4.492366e-09,4.606610e-02,8.442010e-03,1.927734e-04,2.578130e-01,0.999998,0.015886,4.759189e-03,5.268373e-06,0.007195
4,0.001587,5.941178e-20,1.011454e-06,2.331143e-16,1.183358e-26,5.207194e-12,1.411023e-10,6.604779e-17,1.174901e-08,5.334097e-07,5.148958e-14,1.000000e+00,1.000000,0.000043,1.486742e-23,1.821220e-14,0.005468
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61186,0.000016,1.478449e-18,6.998911e-12,6.966736e-15,1.195872e-22,2.836506e-07,5.680908e-04,4.662264e-15,8.965706e-08,2.308274e-03,9.305848e-10,9.995030e-01,0.999361,0.005689,5.435718e-14,4.671766e-17,0.001266
61187,0.272188,4.144245e-11,1.479785e-05,2.337008e-11,1.743999e-19,9.999988e-01,2.505351e-10,1.234076e-05,1.910430e-02,2.968843e-01,9.881233e-09,5.320469e-09,0.999962,0.038111,3.295826e-16,1.869317e-06,0.999957
61188,0.096333,5.685841e-13,2.543295e-06,2.667359e-03,1.053363e-04,9.995882e-01,2.429496e-08,2.799974e-14,3.564263e-03,2.472836e-04,8.454814e-06,2.854007e-04,0.999999,0.025155,2.911590e-03,2.245361e-08,0.080718
61189,0.000721,3.973250e-13,1.798302e-09,5.675274e-14,4.834003e-17,1.371842e-03,9.092738e-01,4.776070e-14,3.300467e-05,1.692611e-04,1.205341e-01,4.062865e-06,0.078661,0.002268,2.653522e-10,5.538999e-17,0.006967


In [26]:
#saving my final result into a csv file
preds = []
for i in tqdm(range(sub.shape[0]), miniters=1000):
    a = sub.loc[[i]]
    a = a.apply(lambda x: x > 0.2, axis=1)
    a = a.transpose()
    a= a.loc[a[i] == True]
    ' '.join(list(a.index))
    preds.append(' '.join(list(a.index)))

sam_sub['tags'] = preds
sam_sub.to_csv('subm.csv', index=False)

100%|██████████| 61191/61191 [02:11<00:00, 466.01it/s]


In [27]:
submitted

,image_name,tags
0,test_0,clear primary
1,test_1,clear primary water
2,test_2,partly_cloudy primary
3,test_3,agriculture clear partly_cloudy primary
4,test_4,partly_cloudy primary
...,...,...
61186,file_9995,partly_cloudy primary
61187,file_9996,agriculture clear habitation primary water
61188,file_9997,clear primary
61189,file_9998,cloudy
